In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __builtin__ import any as b_any

import math
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import numpy as np
from PIL import Image
from alexnet import AlexNet

import tensorflow as tf

import configuration
import inference_wrapper
import sys
sys.path.insert(0, '/Users/ALP/PycharmProjects/MemeProject/im2txt/inference_utils')
import caption_generator
import vocabulary

/Users/ALP/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
np.random.seed(2)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'Jmemes')


FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_string("checkpoint_path", "trainlogIncNEW",
                       "Model checkpoint file or directory containing a "
                       "model checkpoint file.")
tf.flags.DEFINE_string("vocab_file", "vocab4.txt", "Text file containing the vocabulary.")
tf.flags.DEFINE_string("input_files", "Jmemes",
                       "File pattern or comma-separated list of file patterns "
                       "of image files.")

tf.logging.set_verbosity(tf.logging.INFO)


In [ ]:
g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(configuration.ModelConfig(),
                                               FLAGS.checkpoint_path)
g.finalize()


In [ ]:
with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)
    #print(tf.trainable_variables())
    var = [v for v in tf.trainable_variables() if v.name == "seq_embedding/map:0"][0]
    embeddings = sess.run(var)

In [ ]:
em_orig = np.loadtxt('embedding_matrix5')
for i,ting in enumerate(embeddings):
    if em_orig[i] != embeddings[i]:
        print('YESSS')

In [ ]:
np.savetxt('REAL_EMBEDDING_MATRIX',embeddings)

In [ ]:
embeddings[20000,10]

In [ ]:
vocab = vocabulary.Vocabulary(FLAGS.vocab_file)

with open('Captions.txt','r') as f:
    data_captions = f.readlines()
data_captions = [s.lower() for s in data_captions]
  
with open('ordered_memes.txt','r') as f:
    ordered_memes = f.readlines()
ordered_memes = [meme.replace('\n','') for meme in ordered_memes]
#convert jpg image(s) into iamge representations using alexnet:
filenames = [os.path.join(image_dir, f) for f in os.listdir(FLAGS.input_files) if f.endswith('.jpg')]
#filenames = [os.path.join(image_dir, f) for f in ordered_memes[150:160] + ['TutorPP.jpg']]
print(filenames)
tf.logging.info("Running caption generation on %d files matching %s",len(filenames), FLAGS.input_files)


In [ ]:
with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)
    
    # Prepare the caption generator. Here we are implicitly using the default
    # beam search parameters. See caption_generator.py for a description of the
    # available beam search parameters.
    generator = caption_generator.CaptionGenerator(model, vocab)
    num_in_data_total = 0
    num_captions = 0
    for i,filename in enumerate(filenames):
      with tf.gfile.GFile(filename, "rb") as f:
        image = f.read()
      print("Captions for image %s:" % os.path.basename(filenames[i]))
      num_in_data = 0
      for k in range(10):
          captions = generator.beam_search(sess, image)
            
          for i, caption in enumerate(captions):
            # Ignore begin and end words.
            sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
            sentence = " ".join(sentence)
            in_data = 0
            if b_any(sentence in capt for capt in data_captions):
                in_data = 1
                num_in_data += 1
                num_in_data_total += 1
                num_captions += 1
            else:
                num_captions += 1
            print("  %d) %s (p=%f) [in data = %d]" % (i, sentence, math.exp(caption.logprob),in_data))
      print("number of captions in data = %d" % (num_in_data))
    print("(total number of captions in data = %d) percent in data = %f" % (num_in_data_total,(num_in_data_total/num_captions)))

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(g)

In [ ]:
OLD_CHECKPOINT_FILE = "trainlogSGD/model.ckpt-600000"
NEW_CHECKPOINT_FILE = "trainlogSGDNEW/model.ckpt-600000"

import tensorflow as tf
vars_to_rename = {
    "lstm/basic_lstm_cell/weights": "lstm/basic_lstm_cell/kernel",
    "lstm/basic_lstm_cell/biases": "lstm/basic_lstm_cell/bias",
}
new_checkpoint_vars = {}
reader = tf.train.NewCheckpointReader(OLD_CHECKPOINT_FILE)
for old_name in reader.get_variable_to_shape_map():
  if old_name in vars_to_rename:
    new_name = vars_to_rename[old_name]
  else:
    new_name = old_name
  new_checkpoint_vars[new_name] = tf.Variable(reader.get_tensor(old_name))

init = tf.global_variables_initializer()
saver = tf.train.Saver(new_checkpoint_vars)

with tf.Session() as sess:
  sess.run(init)
  saver.save(sess, NEW_CHECKPOINT_FILE)